# Volume 3: MongoDB.
    <Name>
    <Class>
    <Date>

In [87]:
import re
import json
from pymongo import MongoClient

## Problem 1

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

Create a MongoDB database and initialize a collection in the database.
Use the collection's `delete_many()` method with an empy set as input to clear existing contents of the collection, then fill the collection one line at a time with the data from `trump.json`.
Check that your collection has 95,643 entries with its `count()` method.

In [2]:
client = MongoClient()
print("connected")
db = client.db1
col = db.collection1
col.delete_many({})
with open("trump.json", 'r') as f:
    data = f.readlines()
for d in data:
    trump = json.loads(d)
    col.insert_one(trump)

connected


In [4]:
col.estimated_document_count()

95643

## Problem 2

Query the Twitter collection for the following information.
- How many tweets include the word Russia? Use `re.IGNORECASE`.
- How many tweets came from one of the main continental US time zones? These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.
- How often does each language occur? Construct a dictionary with each language and it’s frequency count.

In [90]:
col.find()[0]

{'_id': ObjectId('5c15cad4fbdc862d5cf462d0'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Fri May 26 19:18:30 +0000 2017',
 'entities': {'hashtags': [{'indices': [112, 117], 'text': 'maga'},
   {'indices': [118, 129], 'text': 'trumptrain'},
   {'indices': [130, 136], 'text': 'trump'}],
  'symbols': [],
  'urls': [{'display_url': 'truthfeed.com/liberal-spin-d…',
    'expanded_url': 'http://truthfeed.com/liberal-spin-democrats-are-saying-that-losing-in-montana-is-a-win/76338/',
    'indices': [88, 111],
    'url': 'https://t.co/t6QJFs0i4S'}],
  'user_mentions': [{'id': 707278892801765377,
    'id_str': '707278892801765377',
    'indices': [3, 17],
    'name': 'TruthFeed News',
    'screen_name': 'TruthFeedNews'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 868184586844549121,
 'id_str': '868184586844549121',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_use

In [91]:
results = col.find({'text': {'$regex': re.compile(r'russia', re.IGNORECASE)}})
results.count()

/Users/Seong-EunCho/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


5841

In [93]:
time_zones = ["Central Time (US & Canada)", "Pacific Time (US & Canada)", "Eastern Time (US & Canada)", "Mountain Time (US & Canada)"]
results = col.find({'user.time_zone': {"$in": time_zones}})
results.count()

/Users/Seong-EunCho/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


33939

In [96]:
lang = col.distinct("lang")
lang_dict = {l: col.find({"lang": l}).count() for l in lang}
lang_dict

/Users/Seong-EunCho/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


{'ar': 30,
 'cs': 13,
 'cy': 2,
 'da': 35,
 'de': 635,
 'el': 10,
 'en': 84919,
 'es': 3027,
 'et': 23,
 'eu': 6,
 'fa': 5,
 'fi': 20,
 'fr': 1235,
 'ht': 21,
 'hu': 6,
 'in': 121,
 'is': 1,
 'it': 423,
 'iw': 1,
 'ja': 17,
 'ko': 2,
 'lt': 8,
 'lv': 2,
 'nl': 212,
 'no': 21,
 'pl': 108,
 'pt': 632,
 'ro': 21,
 'ru': 29,
 'sl': 3,
 'sv': 55,
 'th': 3,
 'tl': 48,
 'tr': 588,
 'uk': 3,
 'und': 1653,
 'ur': 1,
 'vi': 7,
 'zh': 1}

## Problem 3

Query the Twitter collection from Problem 1 for the following information.
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? Use the latitude and longitude point in `"coordinates"`.

In [112]:
results = col.find().sort('user.followers_count', -1)
curr_outputs = []
for r in results[:7]:
    if r['user']['name'] not in curr_outputs:
        print(r['user']['name'])
    curr_outputs.append(r['user']['name'])

CNN
Reuters Top News
Fox News
Wall Street Journal
TIME


In [119]:
results = col.find({"entities.hashtags.4": {"$exists": True}}).sort("entities.hashtags.4.indices.0")
results[0]["entities"]["hashtags"][4]["indices"][0]


23

In [139]:
results = col.find({"coordinates.coordinates": {"$exists": True}}).sort("coordinates.coordinates.0", -1)
results[1]["coordinates"]["coordinates"][0]

28.03908658

## Problem 4

Clean the Twitter collection in the following ways.

- Get rid of the `"retweeted_status"` field in each tweet.
- Update every tweet from someone with at least 1000 followers to include a popular field whose value is True. Report the number of popular tweets.
- (OPTIONAL) The geographical coordinates used before in coordinates.coordinates are turned off for most tweets. But many more have a bounding box around the coordinates in the place field. Update every tweet without coordinates that contains a bounding box so that the coordinates contains the average value of the points that form the bounding box. Make the structure of coordinates the same as the others, so it contains coordinates with a longitude, latitude array and a type, the value of which should be ’Point’.

In [144]:
col.update_many({"retweeted_status": {"$exists": True}}, {'$unset': {"retweeted_status": {"$exists": True}}})
results = col.find({"retweeted_status": {"$exists": True}})
results.count()

/Users/Seong-EunCho/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0

In [153]:
col.update_many({"user.followers_count": {"$gt": 1000}}, {"$set": {'popular': True}})
results = col.find({"popular": {"$exists": True}})
results.count()

/Users/Seong-EunCho/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


32481